In [1]:
!pip install -q crewai langchain langchain_core langchain_community langchain-openai langchain-experimental wikipedia "wikibase-rest-api-client<0.2" mediawikiapi arxiv tavily-python

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 kB 6.5 MB/s et

In [17]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict

class ApplicationIdea(BaseModel):
    project_name: str
    description: str

class DevelopmentOutput(BaseModel):
    feasibility: Dict[str, str]
    design_architecture: Dict[str, str]
    recommended_tools: List[str]
    implementation_plan: Dict[str, str]
    estimated_timeline: Optional[str]
    estimated_cost: Optional[float]
    potential_challenges: Optional[List[str]]
    additional_notes: Optional[str]
    resources: Optional[List[str]]
    contact_information: Optional[str]
    legal_disclaimer: Optional[str]

In [18]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [19]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [24]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.arxiv.tool import ArxivAPIWrapper
from langchain.tools import Tool

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)

    def feasibility_agent(self):
        # Agent 1: Feasibility Analyst
        tools = [
            TavilySearchResults(
                max_results=5,
                search_depth="advanced",
                include_answer=True,
                include_raw_content=True,
            ),
            Tool(
                name="Wikipedia",
                func=WikipediaAPIWrapper().run,
                description="Access Wikipedia articles for information."
            ),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="Access academic papers from Arxiv."
            ),
        ]
        return Agent(
            role="Feasibility Analyst",
            backstory=dedent("""You are an expert in assessing the feasibility of software projects, especially those involving LLMs. You utilize tools like Wikipedia and Arxiv to gather necessary information."""),
            goal=dedent("""Analyze the user's application idea and determine the feasibility of developing the desired LLM application. Provide detailed reasons and recommendations, using the provided tools to support your analysis."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def design_agent(self):
        # Agent 2: Solution Architect
        tools = [
            TavilySearchResults(
                max_results=5,
                search_depth="advanced",
                include_answer=True,
                include_raw_content=True,
            ),
            Tool(
                name="Wikipedia",
                func=WikipediaAPIWrapper().run,
                description="Access Wikipedia articles for information."
            ),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="Access academic papers from Arxiv."
            ),
        ]
        return Agent(
            role="Solution Architect",
            backstory=dedent("""You specialize in designing architectures for applications that leverage LLMs. You frequently consult resources like Wikipedia and Arxiv for the latest design patterns and technologies."""),
            goal=dedent("""Provide a detailed design architecture for the LLM application, including components, data flow, and integrations. Use the provided tools to enhance your design recommendations."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def implementation_agent(self):
        # Agent 3: Implementation Planner
        tools = [
            TavilySearchResults(
                max_results=5,
                search_depth="advanced",
                include_answer=True,
                include_raw_content=True,
            ),
            Tool(
                name="Wikipedia",
                func=WikipediaAPIWrapper().run,
                description="Access Wikipedia articles for information."
            ),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="Access academic papers from Arxiv."
            ),
        ]
        return Agent(
            role="Implementation Planner",
            backstory=dedent("""You provide detailed implementation plans for software projects involving LLMs. You utilize tools like Wikipedia and Arxiv to inform your planning."""),
            goal=dedent("""Create an implementation plan for the LLM application, including timeline, cost estimation, and resource requirements. Use the provided tools to inform your plan."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def output_agent(self):
        # Agent 4: Development Advisor
        tools = [
            TavilySearchResults(
                max_results=5,
                search_depth="advanced",
                include_answer=True,
                include_raw_content=True,
            ),
            Tool(
                name="Wikipedia",
                func=WikipediaAPIWrapper().run,
                description="Access Wikipedia articles for information."
            ),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="Access academic papers from Arxiv."
            ),
        ]
        return Agent(
            role="Development Advisor",
            backstory=dedent("""You combine all the information and provide a comprehensive development output to the user. You leverage tools like Wikipedia and Arxiv to ensure your advice is well-informed."""),
            goal=dedent("""Using the initial application idea, provide a comprehensive development output, including feasibility, design architecture, recommended tools, implementation plan, and other relevant details, matching the DevelopmentOutput schema. Use the provided tools to enhance your recommendations."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )


In [26]:
from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __init__(self):
        pass

    def feasibility_task(self, agent, application_idea: ApplicationIdea):
        return Task(
            description=dedent(f"""
                Analyze the following application idea and determine the feasibility of developing the desired LLM application.
                Provide detailed reasons and recommendations.
                **Use the tools provided to support your analysis.**

                **Application Idea**:

    {application_idea.model_dump_json(indent=2)}

            """),
            agent=agent,
            expected_output="A feasibility analysis with detailed reasons and recommendations.",
        )

    def design_task(self, agent, application_idea: ApplicationIdea):
        return Task(
            description=dedent(f"""
                Provide a detailed design architecture for the following application idea.
                Include components, data flow, and integrations.
                **Use the tools provided to enhance your design recommendations.**

                **Application Idea**:

    {application_idea.model_dump_json(indent=2)}

            """),
            agent=agent,
            expected_output="A design architecture including components, data flow, and integrations.",
        )

    def implementation_task(self, agent, application_idea: ApplicationIdea):
        return Task(
            description=dedent(f"""
                Create an implementation plan for the following application idea.
                Include timeline, cost estimation, and resource requirements.
                **Use the tools provided to inform your plan.**

                **Application Idea**:

    {application_idea.model_dump_json(indent=2)}

            """),
            agent=agent,
            expected_output="An implementation plan including timeline, cost estimation, and resource requirements.",
        )

    def development_output_task(self, agent, application_idea: ApplicationIdea):
        development_output_schema = DevelopmentOutput.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Using the initial application idea, provide a comprehensive development output.
                Include feasibility, design architecture, recommended tools, implementation plan, and other relevant details.
                Provide your output in **JSON format** matching the **DevelopmentOutput** schema.
                **Use the tools provided to ensure your advice is well-informed.**

                **Format**:

        json
{development_output_schema}

                **Application Idea**:

    {application_idea.model_dump_json(indent=2)}

            """),
            agent=agent,
            expected_output=f"The development output in JSON format matching the schema: {development_output_schema}",
        )


In [27]:
import os
from crewai import Crew
from textwrap import dedent

# Assuming the schemas and agents are imported from their respective modules
# from schemas import ApplicationIdea
# from agents import CustomAgents
# from tasks import CustomTasks

class LLMDevelopmentAssistantCrew:
    def __init__(self, project_name, description):
        self.application_idea = ApplicationIdea(
            project_name=project_name,
            description=description
        )
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        feasibility_agent = self.agents.feasibility_agent()
        design_agent = self.agents.design_agent()
        implementation_agent = self.agents.implementation_agent()
        output_agent = self.agents.output_agent()

        # Define tasks
        feasibility_task = self.tasks.feasibility_task(feasibility_agent, self.application_idea)
        design_task = self.tasks.design_task(design_agent, self.application_idea)
        implementation_task = self.tasks.implementation_task(implementation_agent, self.application_idea)
        development_output_task = self.tasks.development_output_task(output_agent, self.application_idea)

        # Create the crew
        crew = Crew(
            agents=[
                feasibility_agent,
                design_agent,
                implementation_agent,
                output_agent,
            ],
            tasks=[
                feasibility_task,
                design_task,
                implementation_task,
                development_output_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the LLM Application Development Assistant")
    print("-------------------------------------------------------")
    project_name = input("Enter the project name: ")
    description = input("Provide a brief description of the project: ")

    assistant = LLMDevelopmentAssistantCrew(
        project_name=project_name,
        description=description
    )
    results = assistant.run()
    print("\n\n########################")
    print("## Here is your development assistant result:")
    print("########################\n")
    print(results)


## Welcome to the LLM Application Development Assistant
-------------------------------------------------------
Enter the project name: LLM for Text Generation and Sentiment Analysis
Provide a brief description of the project: Development of a web-based language model using HuggingFace Transformers, designed to generate text, analyze sentiment, and extract keywords. The project targets developers and AI enthusiasts with a budget of $5000, leveraging existing tools like FastAPI and operating under constraints such as limited training data and low computational resources. The project is expected to be completed by 2024-12-01, with an intermediate level of technical expertise.


# Agent: Feasibility Analyst
## Task: 
                Analyze the following application idea and determine the feasibility of developing the desired LLM application.
                Provide detailed reasons and recommendations.
                **Use the tools provided to support your analysis.**

                **Application Idea**:

    {
  "project_name": "LLM for Text Generation and Sentiment Analysis",
  "description": "Development of a web-based language model using HuggingFace Transformers, designed to generate text, analyze sentiment, and extract keywords. The project targets developers and AI enthusiasts with a budget of $5000, leveraging existing tools like FastAPI and operating under constraints such as limited training data and low computational resources. The project is expected to be completed by 2024-12-01, with an intermediate level of technical expertise."
}




# Agent: Feasibility Analyst
## Thought: I need to analyze the feasibility of developing the proposed LLM a

In [28]:
results

CrewOutput(raw='```json\n{\n  "feasibility": {\n    "technical_feasibility": "The use of HuggingFace Transformers for text generation and sentiment analysis is technically feasible given its widespread adoption in NLP tasks. FastAPI is suitable for deploying the model due to its efficiency and ease of use.",\n    "budget_feasibility": "The project budget of $5000 is reasonable for leveraging pre-trained models and existing frameworks like FastAPI, assuming minimal additional computational resources are required."\n  },\n  "design_architecture": {\n    "model_architecture": "The architecture will involve a pre-trained transformer model from HuggingFace for text generation and sentiment analysis. The model will be fine-tuned on specific datasets to improve performance.",\n    "system_architecture": "The system will be a web-based application using FastAPI to handle API requests. The backend will process text inputs, generate outputs using the transformer model, and return results to the 